In [ ]:
# Libraries needed (pandas is not standard and must be installed in Python)
import requests
import pandas as pd

# Insert your own client ID here
client_id = '2e243d34-57bc-42b4-8095-239991af5353'
# Define endpoint and parameters
endpoint = 'https://frost.met.no/observations/v0.jsonld'
parameters = {
    'sources': 'SN18700,SN90450',
    'elements': 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)',
    'referencetime': '2010-04-01/2010-04-03',
}
# Issue an HTTP GET request
r = requests.get(endpoint, parameters, auth=(client_id,''))
# Extract JSON data
json = r.json()

# Check if the request worked, print out any errors
if r.status_code == 200:
    data = json['data']
    print('Data retrieved from frost.met.no!')
else:
    print('Error! Returned status code %s' % r.status_code)
    print('Message: %s' % json['error']['message'])
    print('Reason: %s' % json['error']['reason'])



Data retrieved from frost.met.no!


In [22]:
df_list = []

for item in data:
    row = pd.DataFrame(item['observations'])
    row['referenceTime'] = item['referenceTime']
    row['sourceId'] = item['sourceId']
    df_list.append(row)

# Efficiently concatenate all DataFrames at once
df = pd.concat(df_list, ignore_index=True)

df.head()

,elementId,value,unit,level,timeOffset,timeResolution,timeSeriesId,performanceCategory,exposureCategory,qualityCode,referenceTime,sourceId
0,mean(air_temperature P1D),0.2,degC,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,P1D,0,C,2,0.0,2018-04-01T00:00:00.000Z,SN18700:0
1,mean(air_temperature P1D),-0.7,degC,"{'levelType': 'height_above_ground', 'unit': '...",PT6H,P1D,0,C,2,2.0,2018-04-01T00:00:00.000Z,SN18700:0
2,sum(precipitation_amount P1D),0.0,mm,NaN,PT18H,P1D,0,C,2,2.0,2018-04-01T00:00:00.000Z,SN18700:0
3,sum(precipitation_amount P1D),0.0,mm,NaN,PT6H,P1D,0,C,2,0.0,2018-04-01T00:00:00.000Z,SN18700:0
4,mean(wind_speed P1D),2.3,m/s,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,P1D,0,C,2,2.0,2018-04-01T00:00:00.000Z,SN18700:0
